In [1]:
public class Company
{
    public string Name { get; set; }
    public List<Employee> Employees { get; set; } = new List<Employee>();
    public List<Order> Orders { get; set; } = new List<Order>();

    public Company(string name)
    {
        Name = name;
    }

    public void AssignEmployeeToOrder(Order order, Employee employee)
    {
        order.AssignedEmployee = employee;
    }

    public void UpdateOrderStatus(Order order, OrderStatus status)
    {
        order.Status = status;
        OnOrderStatusChanged(order, status);
    }

    protected virtual void OnOrderStatusChanged(Order order, OrderStatus status)
    {
        StatusChanged?.Invoke(this, new OrderEventArgs(order, status));
    }

    public event EventHandler<OrderEventArgs> StatusChanged;
}

public class Employee
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string Position { get; set; }
    public int OrdersProcessed { get; set; }

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
    }
}

public class Customer
{
    public string FirstName { get; set; }
    public string LastName { get; set; }
    public string PhoneNumber { get; set; }
}

public class Order
{
    public int OrderId { get; private set; }
    public string Description { get; set; }
    public Employee AssignedEmployee { get; set; }
    public OrderStatus Status { get; set; }
    public DateTime CreatedAt { get; private set; }

    // Добавим свойство для доступа к компании
    public Company Company => throw new InvalidOperationException("Order не содержит информации о компании");

    public Order(int orderId, string description)
    {
        OrderId = orderId;
        Description = description;
        CreatedAt = DateTime.Now;
    }
}

public enum OrderStatus
{
    Created,
    InProgress,
    Completed
}
public delegate void OrderStatusHandler(object sender, OrderEventArgs e);

public class OrderEventArgs : EventArgs
{
    public Order Order { get; }
    public OrderStatus NewStatus { get; }

    public OrderEventArgs(Order order, OrderStatus newStatus)
    {
        Order = order;
        NewStatus = newStatus;
    }
}

public interface IOrderProcessor
{
    void ProcessOrder(Order order);
}

public class DefaultOrderProcessor : IOrderProcessor
{
    public void ProcessOrder(Order order)
    {
        Console.WriteLine($"Processing order {order.OrderId}: {order.Description}");
        // Здесь бы могло быть реальная логика обработки заказа
        order.Status = OrderStatus.InProgress;
        order.AssignedEmployee.OrdersProcessed++;
        
        // Уведомление о статусе заказа
        order.Company.OnOrderStatusChanged?.Invoke(order, order.Status);
    }
}
var company = new Company("МegaCorp");
        
        var employee1 = new Employee("Иван", "Иванов", "Менеджер");
        var employee2 = new Employee("Петр", "Петров", "Разработчик");

        company.Employees.Add(employee1);
        company.Employees.Add(employee2);

        var customer = new Customer { FirstName = "Анна", LastName = "Сидорова", PhoneNumber = "+7(495)1234567" };

        for (int i = 1; i <= 5; i++)
        {
            var order = new Order(i, $"Заказ номер {i}");
            order.Company = company;
            order.Customer = customer;

            Console.WriteLine($"Создан заказ {order.OrderId}: {order.Description}");

            // Назначение сотрудника на обработку заказа
            company.AssignEmployeeToOrder(order, employee1);

            // Обработка заказа
            var processor = new DefaultOrderProcessor();
            processor.ProcessOrder(order);

            // Изменение статуса заказа
            company.UpdateOrderStatus(order, OrderStatus.Completed);

            Console.WriteLine($"Статус заказа {order.OrderId}: {order.Status}");
        }

        // Отчет по сотрудникам
        foreach (var emp in company.Employees)
        {
            Console.WriteLine($"{emp.FirstName} {emp.LastName} ({emp.Position}): Обработано заказов - {emp.OrdersProcessed}");
        }

Error: (123,13): error CS0200: Невозможно присвоить значение свойству или индексатору "Order.Company" — доступ только для чтения.
(124,19): error CS1061: "Order" не содержит определения "Customer", и не удалось найти доступный метод расширения "Customer", принимающий тип "Order" в качестве первого аргумента (возможно, пропущена директива using или ссылка на сборку).
(107,23): error CS0122: "Company.OnOrderStatusChanged(Order, OrderStatus)" недоступен из-за его уровня защиты.